In this notebook, we will convert Tensorflow weights of DeepID model to Keras format.

DeepID2 model: https://arxiv.org/pdf/1406.4773.pdf

Pre-trained TensorFlow weights: https://github.com/Ruoyiran/DeepID/tree/master/model

In [1]:
import tensorflow as tf
import keras
from keras.models import Model, Sequential
from keras.layers import Conv2D, Activation, Input, Add
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


Build DeepID Keras model

In [2]:
def create_deepid_keras_model():
    
    myInput = Input(shape=(55, 47, 3))
    
    x = Conv2D(20, (4, 4), name='Conv1', activation='relu', input_shape=(55, 47, 3))(myInput)
    x = MaxPooling2D(pool_size=2, strides=2, name='Pool1')(x)
    x = Dropout(rate=1, name='D1')(x)
    
    x = Conv2D(40, (3, 3), name='Conv2', activation='relu')(x)
    x = MaxPooling2D(pool_size=2, strides=2, name='Pool2')(x)
    x = Dropout(rate=1, name='D2')(x)
    
    x = Conv2D(60, (3, 3), name='Conv3', activation='relu')(x)
    x = MaxPooling2D(pool_size=2, strides=2, name='Pool3')(x)
    x = Dropout(rate=1, name='D3')(x)
    
    #--------------------------------------
    
    x1 = Flatten()(x)
    fc11 = Dense(160, name = 'fc11')(x1)
    
    #--------------------------------------
    
    x2 = Conv2D(80, (2, 2), name='Conv4', activation='relu')(x)
    x2 = Flatten()(x2)
    fc12 = Dense(160, name = 'fc12')(x2)
    
    #--------------------------------------
    
    y = Add()([fc11, fc12])
    y = Activation('relu', name = 'deepid')(y)
    
    model = Model(inputs=[myInput], outputs=y)
    
    return model

In [3]:
deepid_keras_model = create_deepid_keras_model()

In [14]:
deepid_keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 55, 47, 3)    0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 52, 44, 20)   980         input_1[0][0]                    
__________________________________________________________________________________________________
Pool1 (MaxPooling2D)            (None, 26, 22, 20)   0           Conv1[0][0]                      
__________________________________________________________________________________________________
D1 (Dropout)                    (None, 26, 22, 20)   0           Pool1[0][0]                      
__________________________________________________________________________________________________
Conv2 (Con

# Convert TF weights to Keras

A Pretty Guide to convert TensorFlow weights to Keras: https://github.com/nyoki-mtl/keras-facenet/blob/master/notebook/tf_to_keras.ipynb

In [5]:
import os
import re
import numpy as np

In [6]:
tf_model_dir = 'tf-model/'
npy_weights_dir = 'keras-model/npy_weights/'
weights_dir = 'keras-model/weights/'
model_dir = 'keras-model/model/'

weights_filename = 'deepid_keras_weights.h5'

In [7]:
os.makedirs(npy_weights_dir, exist_ok=True)
os.makedirs(weights_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [8]:
re_repeat = re.compile(r'Repeat_[0-9_]*b')
re_block8 = re.compile(r'Block8_[A-Za-z]')

def get_filename(key):
    filename = str(key)
    filename = filename.replace('/', '_')
    filename = re_repeat.sub('B', filename)

    # from TF to Keras naming
    filename = filename.replace('_weights', '_kernel')
    filename = filename.replace('_biases', '_bias')

    return filename + '.npy'

In [9]:
def extract_tensors_from_checkpoint_file(filename, output_folder):
    reader = tf.train.NewCheckpointReader(filename)

    for key in reader.get_variable_to_shape_map():
        # not saving the following tensors
        if key == 'global_step':
            continue
        if 'AuxLogit' in key:
            continue

        # convert tensor name into the corresponding Keras layer weight name and save
        #print("key: ", key)
        path = os.path.join(output_folder, get_filename(key))
        arr = reader.get_tensor(key)
        np.save(path, arr)

TensorFlow weights exist in this repo: https://github.com/Ruoyiran/DeepID/tree/master/model

In [10]:
extract_tensors_from_checkpoint_file(tf_model_dir+'best_model.ckpt', npy_weights_dir)

In [11]:
print('Loading numpy weights from', npy_weights_dir)

for layer in deepid_keras_model.layers:
    
    if layer.weights:
        weights = []
        
        for w in layer.weights:
            weight_name = os.path.basename(w.name).replace(':0', '')
            
            if isinstance(layer, keras.layers.Conv2D):
                layer_name = layer.name + "_conv2d"
            if isinstance(layer, keras.layers.Dense):
                layer_name = "DeepID_" + layer.name
            
            weight_file = layer_name + '_' + weight_name + '.npy'
            
            print(weight_file)
            
            weight_arr = np.load(os.path.join(npy_weights_dir, weight_file))
            weights.append(weight_arr)

        layer.set_weights(weights)

Loading numpy weights from keras-model/npy_weights/
Conv1_conv2d_kernel.npy
Conv1_conv2d_bias.npy
Conv2_conv2d_kernel.npy
Conv2_conv2d_bias.npy
Conv3_conv2d_kernel.npy
Conv3_conv2d_bias.npy
Conv4_conv2d_kernel.npy
Conv4_conv2d_bias.npy
DeepID_fc11_kernel.npy
DeepID_fc11_bias.npy
DeepID_fc12_kernel.npy
DeepID_fc12_bias.npy


In [13]:
deepid_keras_model.save_weights(os.path.join(weights_dir, weights_filename))